In [1]:
# Install RDKit. Takes 2-3 minutes
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

--2020-04-18 11:19:37--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2020-04-18 11:19:37--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  81.12M   126MB/s    in 0.6s    

2020-04-18 11:19:38 (126 MB/s) - ‘Miniconda3-latest-Linux

In [1]:
# import sys
# sys.path.append('/usr/local/lib/python3.7/site-packages/')
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors

ModuleNotFoundError: No module named 'rdkit'

In [52]:
!pip install git+https://github.com/samoturk/mol2vec;

  Cloning https://github.com/samoturk/mol2vec to /tmp/pip-req-build-c17kzywr
  Running command git clone -q https://github.com/samoturk/mol2vec /tmp/pip-req-build-c17kzywr
  Created wheel for mol2vec: filename=mol2vec-0.1-py3-none-any.whl size=14026 sha256=5e980a4fdd4050d76b215baee36139f094c8cdcd7dc0152863979c53c50f3ed3
  Stored in directory: /tmp/pip-ephem-wheel-cache-gp7jxilt/wheels/80/d5/e1/90dc211e4a17780f99eaec43328470a70284cd731ef2a77034
Successfully built mol2vec


In [0]:
import pandas as pd
import numpy as np
import cv2
%matplotlib inline

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [17]:
dataset= pd.read_csv('/gdrive/My Drive/train.csv')
# dataset.head()
test_data= pd.read_csv('/gdrive/My Drive/test.csv')
test_data.head()

,SMILES sequence,Binding Affinity
0,Cc1ccc(C2CNCCN2C)cc1,NaN
1,CCOC(CO)c1ccccc1,NaN
2,CC(=O)Nc1cnn(C)n1,NaN
3,CCC(C)NCc1ncccn1,NaN
4,CC(C)=C1CC(N)C1,NaN


In [0]:
dataset['molecules'] = dataset['SMILES sequence'].apply(lambda x: Chem.MolFromSmiles(x)) 

test_data['molecules']= test_data['SMILES sequence'].apply(lambda x: Chem.MolFromSmiles(x)) 

In [0]:
dataset['molecules'] = dataset['molecules'].apply(lambda x: Chem.AddHs(x))
dataset['num_of_atoms'] = dataset['molecules'].apply(lambda x: x.GetNumAtoms())
dataset['num_of_heavy_atoms'] = dataset['molecules'].apply(lambda x: x.GetNumHeavyAtoms())

test_data['molecules'] = test_data['molecules'].apply(lambda x: Chem.AddHs(x))
test_data['num_of_atoms'] = test_data['molecules'].apply(lambda x: x.GetNumAtoms())
test_data['num_of_heavy_atoms'] = test_data['molecules'].apply(lambda x: x.GetNumHeavyAtoms())

In [0]:
def number_of_atoms(atom_list, data):
    for i in atom_list:
        data['num_of_{}_atoms'.format(i)] = data['molecules'].apply(lambda x: len(x.GetSubstructMatches(Chem.MolFromSmiles(i))))

number_of_atoms(['C','O', 'N', 'Cl'], dataset)
number_of_atoms(['C','O', 'N', 'Cl'], test_data)

In [21]:
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split

#Leave only features columns
train_df = dataset.drop(columns=['SMILES sequence', 'molecules', 'Binding Affinity'])
y = dataset['Binding Affinity'].values

print(train_df.columns)

X_train= train_df
y_train= y
X_test= test_data.drop(columns=['SMILES sequence', 'molecules', 'Binding Affinity'])
print(X_test.columns)

# X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size=.2, random_state=1)

Index(['num_of_atoms', 'num_of_heavy_atoms', 'num_of_C_atoms',
       'num_of_O_atoms', 'num_of_N_atoms', 'num_of_Cl_atoms'],
      dtype='object')
Index(['num_of_atoms', 'num_of_heavy_atoms', 'num_of_C_atoms',
       'num_of_O_atoms', 'num_of_N_atoms', 'num_of_Cl_atoms'],
      dtype='object')


In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
def evaluation(model, X_test, y_test):
    prediction = model.predict(X_test)
    return prediction
    # mae = mean_absolute_error(y_test, prediction)
    # mse = mean_squared_error(y_test, prediction)
    
    # plt.figure(figsize=(15, 10))
    # plt.plot(prediction[:300], "red", label="prediction", linewidth=1.0)
    # plt.plot(y_test[:300], 'green', label="actual", linewidth=1.0)
    # plt.legend()
    # plt.ylabel('logP')
    # plt.title("MAE {}, MSE {}".format(round(mae, 4), round(mse, 4)))
    # plt.show()
    
    # print('MAE score:', mae)
    # print('MSE score:', mse)

In [0]:
#Train the model
ridge = RidgeCV(cv=50)
ridge.fit(X_train, y_train)
#Evaluate results
y_test= y_train
predictions= evaluation(ridge, X_test, y_test)

In [0]:
dataset['tpsa'] = dataset['molecules'].apply(lambda x: Descriptors.TPSA(x))
dataset['mol_w'] = dataset['molecules'].apply(lambda x: Descriptors.ExactMolWt(x))
dataset['num_valence_electrons'] = dataset['molecules'].apply(lambda x: Descriptors.NumValenceElectrons(x))
dataset['num_heteroatoms'] = dataset['molecules'].apply(lambda x: Descriptors.NumHeteroatoms(x))



test_data['tpsa'] = test_data['molecules'].apply(lambda x: Descriptors.TPSA(x))
test_data['mol_w'] = test_data['molecules'].apply(lambda x: Descriptors.ExactMolWt(x))
test_data['num_valence_electrons'] = test_data['molecules'].apply(lambda x: Descriptors.NumValenceElectrons(x))
test_data['num_heteroatoms'] = test_data['molecules'].apply(lambda x: Descriptors.NumHeteroatoms(x))

In [39]:
train_df = dataset.drop(columns=['SMILES sequence', 'molecules', 'Binding Affinity'])
y = dataset['Binding Affinity'].values

print(train_df.columns)

X_train= train_df
y_train= y
X_test= test_data.drop(columns=['SMILES sequence', 'molecules', 'Binding Affinity'])
test_df= X_test

# X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size=.1, random_state=1)

Index(['num_of_atoms', 'num_of_heavy_atoms', 'num_of_C_atoms',
       'num_of_O_atoms', 'num_of_N_atoms', 'num_of_Cl_atoms', 'tpsa', 'mol_w',
       'num_valence_electrons', 'num_heteroatoms'],
      dtype='object')


In [0]:
#Train the model
ridge = RidgeCV(cv=5)
ridge.fit(X_train, y_train)
#Evaluate results and plot predictions
predictions= evaluation(ridge, X_test, y_test)

In [0]:
dataset_word= pd.read_csv('/gdrive/My Drive/train.csv')
dataset_word.head()

test_data_word= pd.read_csv('/gdrive/My Drive/test.csv')

In [0]:
target = dataset_word['Binding Affinity']
dataset_word.drop(columns='Binding Affinity',inplace=True)

In [0]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('/gdrive/My Drive/model_300dim.pkl')

In [0]:
#Transforming SMILES to MOL
dataset_word['molecules'] = dataset_word['SMILES sequence'].apply(lambda x: Chem.MolFromSmiles(x))

test_data_word['molecules'] = test_data_word['SMILES sequence'].apply(lambda x: Chem.MolFromSmiles(x))

In [45]:
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
from gensim.models import word2vec
print('Molecular sentence:', mol2alt_sentence(dataset_word['molecules'][1], radius=1))
print('\nMolSentence object:', MolSentence(mol2alt_sentence(dataset_word['molecules'][1], radius=1)))
print('\nDfVec object:',DfVec(sentences2vec(MolSentence(mol2alt_sentence(dataset_word['molecules'][1], radius=1)), model, unseen='UNK')))

print('Molecular sentence:', mol2alt_sentence(test_data_word['molecules'][1], radius=1))
print('\nMolSentence object:', MolSentence(mol2alt_sentence(test_data_word['molecules'][1], radius=1)))
print('\nDfVec object:',DfVec(sentences2vec(MolSentence(mol2alt_sentence(test_data_word['molecules'][1], radius=1)), model, unseen='UNK')))


Molecular sentence: ['2246728737', '3975275337', '864674487', '903112553', '847961216', '2204949651', '2246699815', '1054767590', '864942730', '1510328189', '3217380708', '2994748777', '3218693969', '3777168895', '2041434490', '3118255683', '3218693969', '725322217', '2041434490', '3118255683', '3218693969', '3777168895']

MolSentence object: MolSentence with 22 words

DfVec object: (22, 100) dimensional vector
Molecular sentence: ['2246728737', '3542456614', '2245384272', '3994088662', '864674487', '2814583100', '2245273601', '602889953', '2245384272', '4022716898', '864662311', '1535166686', '3217380708', '3579962709', '3218693969', '951226070', '3218693969', '98513984', '3218693969', '98513984', '3218693969', '98513984', '3218693969', '951226070']

MolSentence object: MolSentence with 24 words

DfVec object: (24, 100) dimensional vector


In [46]:
#Constructing sentences
dataset_word['sentence'] = dataset_word.apply(lambda x: MolSentence(mol2alt_sentence(x['molecules'], 1)), axis=1)

test_data_word['sentence'] = test_data_word.apply(lambda x: MolSentence(mol2alt_sentence(x['molecules'], 1)), axis=1)

#Extracting embeddings to a numpy.array
#Note that we always should mark unseen='UNK' in sentence2vec() so that model is taught how to handle unknown substructures
dataset_word['mol2vec'] = [DfVec(x) for x in sentences2vec(dataset_word['sentence'], model, unseen='UNK')]
X = np.array([x.vec for x in dataset_word['mol2vec']])
y = target.values

test_data_word['mol2vec'] = [DfVec(x) for x in sentences2vec(test_data_word['sentence'], model, unseen='UNK')]
X_test = np.array([x.vec for x in test_data_word['mol2vec']])
y_test = target.values


X.shape

(9000, 100)

In [0]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=1)
X_train= X
y_train= y
ridge = RidgeCV(cv=5)
ridge.fit(X_train, y_train)
predictions= evaluation(ridge, X_test, y_test)

In [0]:
dataset_word = pd.DataFrame(X)
new_df = pd.concat((dataset_word, train_df), axis=1)

test_data_word = pd.DataFrame(X_test)
new_testdf= pd.concat((test_data_word, test_df), axis=1)

In [0]:
# X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size=.1, random_state=1)
X_train= new_df
X_test= new_testdf
ridge = RidgeCV(cv=5)
ridge.fit(X_train, y_train)
predictions= evaluation(ridge, X_test, y_test)

In [50]:
result= pd.DataFrame({'result' : predictions})
print (predictions)

[-21.19552076 -15.70844217 -23.65608573 ... -22.04269733 -14.94374119
 -23.23062353]


In [52]:
for value in predictions:
  print (value)

-21.195520761480008
-15.70844217174427
-23.65608572818174
-20.212087109473227
-20.806880403609487
-20.55992178858295
-22.675619118921933
-20.436465363396852
-15.33167982826085
-17.639713552122057
-19.45456744323366
-16.487506353727593
-20.282207197432843
-23.646982218872555
-21.25937833643191
-20.478462084991836
-23.76330625574911
-23.57714147019613
-22.298789952921762
-19.721857452458668
-23.416768968078866
-16.18373667320626
-21.688464261311868
-20.49590773403801
-19.03397742566666
-19.296656954237402
-18.04285169860022
-18.646774372530455
-19.164806501447078
-22.911014125542817
-19.680447265199806
-16.369042533640386
-19.996940929292602
-23.423993767930572
-21.912624710473384
-18.69764627027895
-19.229988222881175
-21.047142054359586
-14.788130243346846
-19.475124790941162
-19.931388132066534
-22.13003555489561
-15.638979337697062
-16.77969390415419
-18.54878185609925
-22.048548720749466
-23.26888125107224
-23.007017717608417
-16.971673467939976
-24.737531030471676
-20.9859962842449

In [62]:
df= pd.DataFrame({'Binding Affinity' : predictions})
# df.to_csv('submission.txt')

from google.colab import files
# files.download('submission.csv')

NameError: ignored